 1. Обучить любую модель классификации на датасете IRIS до применения PCA и после него. Сравнить качество классификации по отложенной выборке.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# Загрузим игрушечный датасет из sklearn
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Для начала отмасштабируем выборку
X_ = X.astype(float)

rows, cols = X_.shape

# центрирование - вычитание из каждого значения среднего по строке
means = X_.mean(0)
for i in range(rows):
    for j in range(cols):
        X_[i, j] -= means[j]

# деление каждого значения на стандартное отклонение
std = np.std(X_, axis=0)
for i in range(cols):
    for j in range(rows):
        X_[j][i] /= std[i]
        
# Найдем собственные векторы и собственные значения (англ. Eigenvalues)
 
covariance_matrix = X_.T.dot(X_)

eig_values, eig_vectors = np.linalg.eig(covariance_matrix)

# сформируем список кортежей (собственное значение, собственный вектор)
eig_pairs = [(np.abs(eig_values[i]), eig_vectors[:, i]) for i in range(len(eig_values))]

# и отсортируем список по убыванию собственных значений
eig_pairs.sort(key=lambda x: x[0], reverse=True)

print('Собственные значения в порядке убывания:')
for i in eig_pairs:
    print(i)

Собственные значения в порядке убывания:
(437.7746724797989, array([ 0.52106591, -0.26934744,  0.5804131 ,  0.56485654]))
(137.1045707202105, array([-0.37741762, -0.92329566, -0.02449161, -0.06694199]))
(22.013531335697213, array([-0.71956635,  0.24438178,  0.14212637,  0.63427274]))
(3.107225464292848, array([ 0.26128628, -0.12350962, -0.80144925,  0.52359713]))


In [3]:
eig_sum = sum(eig_values)
var_exp = [(i / eig_sum) * 100 for i in sorted(eig_values, reverse=True)]
cum_var_exp = np.cumsum(var_exp)
print(f'Доля дисперсии, описываемая каждой из компонент \n{var_exp}')

# а теперя оценим кумулятивную (то есть накапливаемую) дисперсию при учитывании каждой из компонент
print(f'Кумулятивная доля дисперсии по компонентам \n{cum_var_exp}')

Доля дисперсии, описываемая каждой из компонент 
[72.96244541329989, 22.850761786701774, 3.668921889282873, 0.5178709107154752]
Кумулятивная доля дисперсии по компонентам 
[ 72.96244541  95.8132072   99.48212909 100.        ]


In [4]:
# Сформируем вектор весов из собственных векторов, соответствующих первым двум главным компонентам
W = np.hstack((eig_pairs[0][1].reshape(4,1), eig_pairs[1][1].reshape(4,1)))

print(f'Матрица весов W:\n', W)

Матрица весов W:
 [[ 0.52106591 -0.37741762]
 [-0.26934744 -0.92329566]
 [ 0.5804131  -0.02449161]
 [ 0.56485654 -0.06694199]]


In [5]:
# Сформируем новую матрицу "объекты-признаки"
Z = X_.dot(W)

In [6]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
XPCAreduced = pca.fit_transform(X_)

In [7]:
# Евклидова метрика
def e_metrics(x1, x2):
    
    distance = 0
    for i in range(len(x1)):
        distance += np.square(x1[i] - x2[i])
    
    return np.sqrt(distance)

# Точность
def accuracy(pred, y):
    return (sum(pred == y) / len(y))

In [8]:
# Алгоритм KNN с добавлением весов 𝑤(i)=𝑞**i ,  𝑞∈(0,1) ;
def knn_wi(x_train, y_train, x_test, k, q):
    
    answers = []
    for x in x_test:
        test_distances = []
            
        for i in range(len(x_train)):
            
            # расчет расстояния от классифицируемого объекта до
            # объекта обучающей выборки
            distance = e_metrics(x, x_train[i])
            
            # Записываем в список значение расстояния и ответа на объекте обучающей выборки
            test_distances.append((distance, y_train[i]))
        
        # создаем словарь со всеми возможными классами
        classes = {class_item: 0 for class_item in set(y_train)}
        
        # Сортируем список и среди первых k элементов подсчитаем частоту появления разных классов
        for j, d in enumerate(sorted(test_distances)[0:k]):
            wi = q**j
            classes[d[1]] += wi
            
        # Записываем в список ответов класс с наибольшим весом
        answers.append(sorted(classes, key=classes.get)[-1])
    return answers

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2, random_state=1)

In [10]:
%%time
k, q = 3, 0.9
y_pred = knn_wi(X_train, y_train, X_test, k, q)
print(f'Точность алгоритма при k = {k}, q = {q}: {accuracy(y_pred, y_test):.3f}')

Точность алгоритма при k = 3, q = 0.9: 1.000
Wall time: 48 ms


In [11]:
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(XPCAreduced, y, test_size=0.2, random_state=1)

In [12]:
%%time
y_pred_pca = knn_wi(X_train_pca, y_train_pca, X_test_pca, k, q)
print(f'Точность алгоритма при k = {k}, q = {q}: {accuracy(y_pred_pca, y_test_pca):.3f}')

Точность алгоритма при k = 3, q = 0.9: 0.967
Wall time: 28 ms


Видим, что потеря качества небольшая (3,3%), а вычисления быстрее почти в 2 раза.

 2*. Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()

In [13]:
def SVD(X):
    u, s, vh = np.linalg.svd(X, full_matrices=True)
    W = vh.transpose()[:, :len(s)]
    Z = np.dot(X, W)
    return Z

In [15]:
Z = SVD(X_)
Z[:10]

array([[-2.26470281, -0.4800266 ,  0.12770602,  0.0241682 ],
       [-2.08096115,  0.67413356,  0.23460885,  0.10300677],
       [-2.36422905,  0.34190802, -0.04420148,  0.02837705],
       [-2.29938422,  0.59739451, -0.09129011, -0.06595556],
       [-2.38984217, -0.64683538, -0.0157382 , -0.03592281],
       [-2.07563095, -1.48917752, -0.02696829,  0.00660818],
       [-2.44402884, -0.0476442 , -0.3354704 , -0.03677556],
       [-2.23284716, -0.22314807,  0.0886955 , -0.0246121 ],
       [-2.33464048,  1.11532768, -0.14507686, -0.02685922],
       [-2.18432817,  0.46901356,  0.25376557, -0.03989929]])